In [3]:
# Naive Bayes On The Iris Dataset

from csv import reader
from random import seed, randrange
from math import sqrt, exp, pi

In [ ]:
#definition des fonctions necessaire

In [2]:
# Load a CSV file

def load_csv(filename):
    dataset = list()

    # Ouvre le fichier CSV en mode lecture ('r')
    with open(filename, 'r') as file:
        # Crée un objet CSV reader pour lire le contenu du fichier
        csv_reader = reader(file)

        # Parcourt chaque ligne du fichier CSV
        for row in csv_reader:
            # Vérifie si la ligne est vide, et passe à la ligne suivante si c'est le cas
            if not row:
                continue
            
            # Ajoute la ligne non vide à la liste dataset
            dataset.append(row)

    # Retourne la liste complète contenant toutes les lignes du fichier CSV
    return dataset

In [3]:
#un code plus simple
def load_csv(filename):
    # Initialiser une liste vide pour stocker les données CSV
    dataset = []

    # Ouvrir le fichier CSV en mode lecture ('r')
    with open(filename, 'r') as file:
        # Créer un objet lecteur CSV pour lire le contenu du fichier
        csv_reader = reader(file)
        
        # ajouter les lignes non vides à la liste dataset
        dataset = [row for row in csv_reader if row]

    # Retourner la liste contenant les lignes non vides du fichier CSV
    return dataset

In [4]:
def str_column_to_float(dataset, column):
    """
    Convertit la colonne de chaînes spécifiée en nombre à virgule flottante dans le jeu de données.

    Args:
    - dataset (list of lists): Le jeu de données.
    - column (int): L'index de la colonne à convertir.
    """
    for row in dataset:
        row[column] = float(row[column].strip())

In [5]:
#code plus simple en utilisation les iteration
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = {value: i for i, value in enumerate(unique)}

    for row in dataset:
        row[column] = lookup[row[column]]

    return lookup

def cross_validation_split(dataset, n_folds):
    dataset_split = [list() for _ in range(n_folds)]
    dataset_copy = list(dataset)
    fold_size = len(dataset) // n_folds

    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split[i] = fold

    return dataset_split

def accuracy_metric(actual, predicted):
    correct = sum(1 for i in range(len(actual)) if actual[i] == predicted[i])
    return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()

    for fold in folds:
        train_set = [row for subset in folds if subset != fold for row in subset]
        test_set = [list(row) for row in fold]
        for row in test_set:
            row[-1] = None

        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)

    return scores

def separate_by_class(dataset):
    separated = dict()
    for vector in dataset:
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = list()
        separated[class_value].append(vector)

    return separated

def mean(numbers):
    return sum(numbers) / float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum((x - avg)**2 for x in numbers) / float(len(numbers) - 1)
    return sqrt(variance)

def summarize_dataset(dataset):
    return [(mean(column), stdev(column), len(column)) for column in zip(*dataset)][:-1]

def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    return {class_value: summarize_dataset(rows) for class_value, rows in separated.items()}

def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean)**2 / (2 * stdev**2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

def calculate_class_probabilities(summaries, row):
    total_rows = sum(summ[0][2] for summ in summaries.values())
    probabilities = {class_value: summ[0][2] / float(total_rows) for class_value, summ in summaries.items()}

    for i, class_summaries in enumerate(summaries.values()):
        for class_value, probability in probabilities.items():
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)

    return probabilities

def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = max(probabilities.items(), key=lambda x: x[1])
    return best_label

def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    return [predict(summarize, row) for row in test]

In [12]:
# Initialisation du random pour des chiffres aleatoire de 1 à 5 pour garantir la reproductibilité des résultats
seed(1)

# Chargement du fichier CSV 'iris.csv' dans dataframe
filename = 'C:\\Users\\Utilisateur2\\Documents\\FORM-DEVIA\\3-CLASSIFICATION\\1Naive bayes classifer\\ws1\\iris.csv'
dataset = load_csv(filename)

# Conversion de chaque colonne de chaînes en nombre à virgule flottante
# Boucle à travers toutes les colonnes sauf la dernière
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

In [16]:
dataset

[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
 [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
 [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
 [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
 [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
 [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
 [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
 [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
 [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
 [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
 [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
 [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
 [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
 [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
 [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
 [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
 [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
 [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
 [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
 [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
 [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
 [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
 [4.6, 3.6, 1.0, 0.2, 'Iris-setosa'],
 [5.1, 3.3, 1.7, 0.5, 'Iris-setosa'],
 [4.8, 3.4, 1.9, 0.2, 'Iris-setosa'],
 [5.0, 3.0, 1.6, 0.2, 'Iris-setosa'],
 [5.0, 3.4, 

In [23]:
import random
# Convertir la colonne de classes en entiers
str_column_to_int(dataset, len(dataset[0]) - 1)

# Évaluer l'algorithme
n_folds = 5
scores = evaluate_algorithm(dataset, naive_bayes, n_folds)

# Afficher les scores obtenus
print('Scores : %s' % scores)

# Afficher la précision moyenne en pourcentage
print('Précision moyenne : %.3f%%' % (sum(scores) / float(len(scores))))

Scores : [20.0, 30.0, 20.0, 26.666666666666668, 33.33333333333333]
Précision moyenne : 26.000%


In [5]:
#irisdataset
# Importation des modules nécessaires
from csv import reader
from math import sqrt, exp, pi

# Charger un fichier CSV
def charger_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convertir une colonne de chaînes en nombres à virgule flottante
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convertir une colonne de chaînes en entiers
def str_column_to_int(dataset, column):
    valeurs_classes = [row[column] for row in dataset]
    uniques = set(valeurs_classes)
    lookup = dict()
    for i, valeur in enumerate(uniques):
        lookup[valeur] = i
        print('[%s] => %d' % (valeur, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Séparer le jeu de données par valeurs de classe, retourne un dictionnaire
def separer_par_classe(dataset):
    separes = dict()
    for i in range(len(dataset)):
        vecteur = dataset[i]
        valeur_classe = vecteur[-1]
        if valeur_classe not in separes:
            separes[valeur_classe] = list()
        separes[valeur_classe].append(vecteur)
    return separes

# Calculer la moyenne d'une liste de nombres
def moyenne(numbers):
    return sum(numbers) / float(len(numbers))

# Calculer l'écart-type d'une liste de nombres
def ecart_type(numbers):
    moy = moyenne(numbers)
    variance = sum([(x - moy)**2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

# Calculer la moyenne, l'écart-type et le nombre d'éléments pour chaque colonne dans un jeu de données
def resumer_jeu_de_donnees(dataset):
    resume = [(moyenne(colonne), ecart_type(colonne), len(colonne)) for colonne in zip(*dataset)]
    del(resume[-1])
    return resume

# Séparer le jeu de données par classe puis calculer les statistiques pour chaque ligne
def resumer_par_classe(dataset):
    separes = separer_par_classe(dataset)
    resume = dict()
    for valeur_classe, lignes in separes.items():
        resume[valeur_classe] = resumer_jeu_de_donnees(lignes)
    return resume

# Calculer la fonction de distribution de probabilité gaussienne pour x
def calculer_probabilite(x, moyenne, ecart_type):
    exponentiel = exp(-((x - moyenne)**2 / (2 * ecart_type**2)))
    return (1 / (sqrt(2 * pi) * ecart_type)) * exponentiel

# Calculer les probabilités de prédire chaque classe pour une ligne donnée
def calculer_probabilites_classe(resumes, ligne):
    total_lignes = sum([resumes[label][0][2] for label in resumes])
    probabilites = dict()
    for valeur_classe, resume_classe in resumes.items():
        probabilites[valeur_classe] = resumes[valeur_classe][0][2] / float(total_lignes)
        for i in range(len(resume_classe)):
            moyenne, ecart_type, _ = resume_classe[i]
            probabilites[valeur_classe] *= calculer_probabilite(ligne[i], moyenne, ecart_type)
    return probabilites

# Prédire la classe pour une ligne donnée
def predire(resumes, ligne):
    probabilites = calculer_probabilites_classe(resumes, ligne)
    meilleure_classe, meilleure_prob = None, -1
    for valeur_classe, probabilite in probabilites.items():
        if meilleure_classe is None or probabilite > meilleure_prob:
            meilleure_prob = probabilite
            meilleure_classe = valeur_classe
    return meilleure_classe

# Faire une prédiction avec Naive Bayes sur le jeu de données Iris
nom_fichier = 'C:\\Users\\Utilisateur2\\Documents\\FORM-DEVIA\\3-CLASSIFICATION\\1Naive bayes classifer\\ws1\\iris.csv'
jeu_de_donnees = charger_csv(nom_fichier)
for i in range(len(jeu_de_donnees[0]) - 1):
    str_column_to_float(jeu_de_donnees, i)

# Convertir la colonne de classe en entiers
str_column_to_int(jeu_de_donnees, len(jeu_de_donnees[0]) - 1)

# Entraîner le modèle
modele = resumer_par_classe(jeu_de_donnees)

# Définir un nouvel enregistrement
ligne = [5.7, 2.9, 4.2, 1.3]

# Prédire l'étiquette
etiquette = predire(modele, ligne)
print('Données=%s, Prédiction : %s' % (ligne, etiquette))

[Iris-versicolor] => 0
[Iris-virginica] => 1
[Iris-setosa] => 2
Données=[5.7, 2.9, 4.2, 1.3], Prédiction : 0


In [7]:
#sur iris data
from csv import reader
from math import sqrt, exp, pi

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if row:
                dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = {value: i for i, value in enumerate(unique)}
    for value, i in lookup.items():
        print('[%s] => %d' % (value, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
    separated = dict()
    for vector in dataset:
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum((x - avg) ** 2 for x in numbers) / float(len(numbers) - 1)
    return sqrt(variance)

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del summaries[-1]
    return summaries

# Split dataset by class then calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {class_value: summarize_dataset(rows) for class_value, rows in separated.items()}
    return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum(summaries[label][0][2] for label in summaries)
    probabilities = {class_value: summaries[class_value][0][2] / float(total_rows) for class_value in summaries}
    for class_value, class_summaries in summaries.items():
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Make a prediction with Naive Bayes on Iris Dataset
filename = 'C:\\Users\\Utilisateur2\\Documents\\FORM-DEVIA\\3-CLASSIFICATION\\1Naive bayes classifer\\ws1\\iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0]) - 1)
# fit model
model = summarize_by_class(dataset)
# define a new record
row = [5.7, 2.9, 4.2, 1.3]
# predict the label
label = predict(model, row)
print('Data=%s, Predicted: %s' % (row, label))

[Iris-versicolor] => 0
[Iris-virginica] => 1
[Iris-setosa] => 2
Data=[5.7, 2.9, 4.2, 1.3], Predicted: 0
